# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [3]:
import numpy as np
import pandas as pd

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [100, 90, 10, 450, 50, 37, 99, 120, 34, 100]
prices_bought = [110, 190, 100, 450]

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [4]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):

    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    prices_recommended =np.array(prices_recommended)
    
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = (flags @ prices_recommended) / sum(prices_bought) 
    
    return recall
    

In [5]:
recall_at_k(recommended_list, bought_list, k=5)

0.5

In [6]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5)

0.6470588235294118

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [7]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
   
    recommended_list = np.array(recommended_list)
    bought_list = np.array(bought_list)

    flags = np.isin(recommended_list, bought_list)

    # если среди рекомендаций не было ни одной покупки
    if (not np.any(flags)):
        return np.inf

    first_index = np.argmax(flags) + 1 # считаем индекс с 1 
    
    result = 1 / first_index

    return result

In [9]:
reciprocal_rank_at_k(recommended_list, bought_list)

1.0

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [10]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    idcg = dcg_at_k(sorted(flags, reverse=True), k)
    if not idcg:
        return 1.  # CHANGE THIS
    return dcg_at_k(flags, k) / idcg

def dcg_at_k(recommended_list, k):
    r = np.asfarray(recommended_list)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.
    

In [11]:
ndcg_at_k(recommended_list, bought_list)

0.8772153153380493